### normal vector 방향으로 random한 error 추가

 - user error를 가장한 plane위의 error를 가진 point를 만들었다 (02)
 
 - 이후 3D bone model 표면 상의 점을 찾아 point를 구했다 (03)
 
 - 이제 이 point를 포함하는 vertex 중 임의의 vertex를 하나 선택하고 이 vector 방향으로 random한 error를 주어야 함
 
 - ref
 
 > https://www.gamedev.net/forums/topic/615947-find-point-along-vector-direction/

In [3]:
import numpy as np

# 다시 읽어드리기

each_point_data = np.load('data/femur_each_point_data.npy')
vertex_data = np.load('data/femur_vertex_data.npy')
vector_data = np.load('data/femur_vector_data.npy')

nearest_points40_with_error_on_plane = np.load('data/nearest_points40_with_error_on_plane.npy')

 - +, - 0.7 mm이내의 random error 생성

In [62]:
import random
import math

In [11]:
error_value = (random.randint(-6, 6) + random.random()) * 0.1
error_value

0.6052413394876274

 - error가 포함된 3D bone 표면 상의 points 데이터를 포함하는 vertex들 구하기

In [16]:
nearest_points40_vertex_group = []

for i in range(len(nearest_points40_with_error_on_plane)):
    print(i)
    temp = []
    for j in range(len(vertex_data)):
        if nearest_points40_with_error_on_plane[i] in vertex_data[j]:
            temp.append(j)
        if j % 10000 == 0:
            print(j)
    nearest_points40_vertex_group.append(temp)

0
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
1
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
2
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
3
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
4
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
5
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
6
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
7
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
8
0
10000
20000
30000
40000
5000

In [ ]:
def find_vertex_and_vector_with_point(points, vertex_data, vector_data):
    """
    input data structure
    points = [[x0, y0, z0], ... [xn, yn, zn]]
    vertex_data = [[[px00, py00, pz00], [px01, py01, pz01], [px02, py02, pz02]],
                    ...,
                   [[pxm0, pym0, pzm0], [pxm1, pym1, pzm1], [pxm2, pym2, pzm2]]]
    vector_data = [[vx0, vy0, vz0], ... [vxm, vym, vzm]]
    
    output data structure
    vertex_data = [[[px00, py00, pz00], [px01, py01, pz01], [px02, py02, pz02]],
                    ...,
                   [[pxn0, pyn0, pzn0], [pxn1, pyn1, pzn1], [pxn2, pyn2, pzn2]]]
    vector_data = [[vx0, vy0, vz0], ... [vxn, vyn, vzn]]
    """
    if len(vertex_data) == len(vector_data):
        return False
    
    group_idx = []
    
    vertex_group = []
    vector_group = []
    
    for i in range(len(points)):
        print(i)
        temp = []
        for j in range(len(vertex_data)):
            if points[i] in vertex_data[j]:
                temp.append(j)
        group_idx.append(temp)
    
    for i in range(len(group_idx)):
        vertex_group.append(vertex_data[group_idx[i]])
        vector_group.append(vector_data[group_idx[i]])
        
    return vertex_group, vector_group

 - point를 포함하는 여러 vertex들 중 하나를 random하게 선택하기
 
 - 동시에 point를 포함하는 여러 vector도 같이 선택 ... vertex가 가진 vector

In [21]:
nearest_points40_vertex_selected = []

for i in range(len(nearest_points40_vertex_group)):
    randomIndex = random.choice(nearest_points40_vertex_group[i])
    nearest_points40_vertex_selected.append(randomIndex)

In [26]:
nearest_points40_vertex = []
nearest_points40_vector = []

for i in range(len(nearest_points40_vertex_selected)):
    nearest_points40_vertex.append(vertex_data[nearest_points40_vertex_selected[i]])
    nearest_points40_vector.append(vector_data[nearest_points40_vertex_selected[i]])

 - vector의 방향에 따라 error를 추가하는 함수 작성

In [77]:
def add_error_to_point_following_to_vector_dirct(point, error_length, vector):
    """
    input data structure
    point = [x, y, z]
    error_length = float (int)
    vector = [unit x vct, unit y vct, unit z vct]
    
    output data structure
    point_added_error_to_vector_direction = [xe, ye, ze]
    """
    point_with_error = []
    unit_error = []
    
    for i in range(len(point)):
        unit_error.append(error_length * vector[i])
    
    for i in range(len(point)):
        point_with_error.append(point[i] + unit_error[i])
    
    return point_with_error

 - vector 방향에 따라 error를 추가한 40개의 point를 생성

In [105]:
points40_with_uncertainty_error = []

for i in range(len(nearest_points40_with_error_on_plane)):
    # point마다 서로 다른 error 값을 추가
    error_value = (random.randint(-6, 6) + random.random()) * 0.1
    tmp_point = add_error_to_point_following_to_vector_dirct(nearest_points40_with_error_on_plane[i],
                                                             error_value,
                                                             nearest_points40_vector[i])
    points40_with_uncertainty_error.append(tmp_point)

###### npy 파일로 저장

In [108]:
import numpy as np

np.save('data/points40_with_uncertainty_error', np.array(points40_with_uncertainty_error))